# Loan Applicant Analyzer with Machine Learning
### Project by: [Demi Oyebanji](mailto:oluwademiladeoyebanji@outlook.com)
This analyzer provides insights into which applications are most likely successfully pay off their loan by training on a large dataset of previously successful or unsuccessful loans

The notebook is formatted for use on [Google Colab](https://colab.research.google.com/)

## Imports

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
# Upload the 'applicants_data.csv' file to train and test the model
from google.colab import files
uploaded = files.upload()

Saving applicants_data.csv to applicants_data.csv


In [3]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv("applicants_data.csv")

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Data Preparation

In [4]:
#Remove columns that have no effect on approval / denial such as Name and EIN
applicant_data_df = applicant_data_df.drop(columns=['NAME', 'EIN'])

applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Convert columns to categoricals where applicable so they can interact with the Machine Learning Model by Encoding them into Binary Values

In [5]:
 categorical_variables = ['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT','SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

print(categorical_variables)

['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']


In [6]:
enc = OneHotEncoder(sparse=False)

encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)

In [8]:
encoded_df.head(5)

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
#Concatinate the new and old data into one DataFrame
encoded_df_con = pd.concat([applicant_data_df, encoded_df], axis=1)
encoded_df_con

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Machine Learning Training

In [10]:
# Define the target so the bot can learn to predict 'IS_SUCCESSFUL'
y = encoded_df_con['IS_SUCCESSFUL']

#Define the features the bot should use to determine the target
X = encoded_df_con.drop(columns=['IS_SUCCESSFUL_1', 'IS_SUCCESSFUL_0', 'IS_SUCCESSFUL', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])


In [11]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the the number of inputs/ outputs
number_input_features = 118

number_output_neurons = 1

hidden_nodes_layer1 = 500

hidden_nodes_layer2 = 35



In [14]:
Sequencer = Sequential()

# Add the first hidden layer with 10 nodes and input dimension of 15
Sequencer.add(Dense(hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Add the second hidden layer with 35 nodes
Sequencer.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer with 3 nodes (assuming it's a classification problem)
Sequencer.add(Dense(number_output_neurons, activation='softmax'))

Sequencer.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               59500     
                                                                 
 dense_1 (Dense)             (None, 35)                17535     
                                                                 
 dense_2 (Dense)             (None, 1)                 36        
                                                                 
Total params: 77071 (301.06 KB)
Trainable params: 77071 (301.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
Sequencer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Fit the model
result = Sequencer.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


632/644 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5313

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


644/644 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 2/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 3/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 4/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 5/50
644/644 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 6/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 7/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_l

## Evaluate the Model

In [20]:
# Evaluate the model on test data
model_loss, model_accuracy = Sequencer.evaluate(X_test, y_test)

# Print the results
print("Model Loss:", model_loss)
print("Model Accuracy:", model_accuracy)



268/268 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.5292
Model Loss: 0.0
Model Accuracy: 0.5292128324508667


In [22]:
# Set the path where the model should be saved to
file_path = "Resources/AlphabetSoup.h5"

# Export the model as a HDF5 file
Sequencer.save(file_path)

# Download the model from colab
files.download(file_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train and evaluate an alternate model

In [16]:
# Define the the number of inputs (features) to the model
alternate_input_features = len(X_train.iloc[0])

number_output_neurons_A1 = 1

hidden_nodes_layer1_A1 = 100
hidden_nodes_layer1_A2 = 80
hidden_nodes_layer1_A3 = 50
hidden_nodes_layer1_A4 = 25
hidden_nodes_layer1_A5 = 5

AlternateSequencer = Sequential()

# Add the first hidden layer
AlternateSequencer.add(Dense(hidden_nodes_layer1_A1, input_dim=alternate_input_features, activation='relu'))

# Add the second hidden layer
AlternateSequencer.add(Dense(hidden_nodes_layer1_A2, activation='relu'))

# Add the third hidden layer
AlternateSequencer.add(Dense(hidden_nodes_layer1_A3, activation='relu'))

# Add the fourth hidden layer
AlternateSequencer.add(Dense(hidden_nodes_layer1_A4, activation='relu'))

# Add the fifth hidden layer
AlternateSequencer.add(Dense(hidden_nodes_layer1_A5, activation='relu'))

# Add the output layer with 3 nodes (assuming it's a classification problem)
AlternateSequencer.add(Dense(number_output_neurons_A1, activation='softmax'))

AlternateSequencer.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               11900     
                                                                 
 dense_4 (Dense)             (None, 80)                8080      
                                                                 
 dense_5 (Dense)             (None, 50)                4050      
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dense_7 (Dense)             (None, 5)                 130       
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 25441 (99.38 KB)
Trainable params: 25441

In [17]:
AlternateSequencer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Fit the model
altresult = AlternateSequencer.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


643/644 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5307

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


644/644 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 2/50
644/644 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 3/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 4/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 5/50
644/644 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 6/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_loss: 0.0000e+00 - val_accuracy: 0.5446
Epoch 7/50
644/644 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5307 - val_l

In [18]:
alt_loss, alt_accuracy = AlternateSequencer.evaluate(X_test, y_test)

268/268 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.5292


## Compare Final Results

In [21]:
print("Original Model Results")
# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

print("Alternate Model Results")
# Display the model loss and accuracy results
print(f"Loss: {alt_loss}, Accuracy: {alt_accuracy}")

Original Model Results
Loss: 0.0, Accuracy: 0.5292128324508667
Alternate Model Results
Loss: 0.0, Accuracy: 0.5292128324508667


In [22]:
# Set the file path for the second model
alt_path = "Resources/AlternateSoup.h5"

# Export the model as an HDF5 file
AlternateSequencer.save(alt_path)

# Download the model to the path
files.download(alt_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>